In [1]:
import numpy
import matplotlib.pyplot as plt

In [2]:
total=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\间隔_5min_init.npy',allow_pickle=True).item()
info=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\路段信息_简略.npy',allow_pickle=True).item()

In [1]:
weekdays_index=[2, 3, 4, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 28, 29, 30, 31, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 49]

In [2]:
weekdays_index_test=[50, 51, 52,55, 56, 57, 58, 59 ]

In [23]:
non_holidays_index=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45 ]

In [21]:
non_holidays_index_test=[49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]

In [22]:
len(non_holidays_index_test)

11

In [4]:
# targets=[54, 56, 58, 316, 45, 246, 42, 38, 52, 118, 24, 23]
targets=[5, 3, 43, 44,46, 138, 34, 32, 297]+[316,59,71,57,55,153,152,221,385,42]
train={}
test={}
for seg in targets:
    train[seg]=[total[seg][i] for i in range(48)]
    test[seg]=[total[seg][i] for i in range(48,60)]

In [5]:
for seg in targets:
    seg_len=float(info[str(seg)][5])
    len_arr=numpy.array([seg_len for i in range(48*180)])
    len_arr=len_arr.reshape((48,180))
    speed=numpy.around(len_arr*3.6/train[seg],2)
    # print(numpy.std(speed))
    print(seg,numpy.around(numpy.mean(speed),2))

5 25.27
3 28.04
43 20.95
44 25.47
46 22.14
138 36.81
34 21.25
32 22.04
297 22.75
316 31.13
59 30.42
71 23.99
57 21.05
55 13.23
153 17.9
152 19.23
221 29.9
385 22.69
42 16.39


In [6]:
def get_ewa(series,look_back_span):
    ewa=[numpy.mean(series[:look_back_span])]
    index=0
    for value in series[look_back_span:]:
        weight=2/(1+value)
        ewa.append(value*weight+ewa[index]*(1-weight))
        index+=1
    return ewa

In [17]:
for seg in targets:
    tmp=[get_ewa(day,4) for day in train[seg]]
    train[seg]=tmp

## 归一化

In [18]:
max_min_time={}

In [19]:
train_days,train_dim=(len(train[targets[0]]),len(train[targets[0]][0]))
test_days,test_dim=(len(test[targets[0]]),len(test[targets[0]][0]))
for seg in targets:
    train_arr=numpy.array(train[seg])
    test_arr=numpy.array(test[seg])
    max_train=numpy.max(train_arr)
    min_train=numpy.min(train_arr) 
    max_min_time[seg]=[max_train,min_train]
    # max_train,min_train=max_min_time[seg]
    bias=(max_train-min_train)

    min_test_arr=[]
    for i in range(test_days):
        min_test_arr.append([min_train]*test_dim)
    min_train_arr=[]
    for i in range(train_days):
        min_train_arr.append([min_train]*train_dim)

    min_train_arr=numpy.array(min_train_arr)#array化
    min_test_arr=numpy.array(min_test_arr)#array化

    train_arr=numpy.around((train_arr-min_train_arr)/bias,2)
    test_arr=numpy.around((test_arr-min_test_arr)/bias,2)

    # train[seg]=train_arr[:32]
    train[seg]=train_arr
    test[seg]=test_arr
    # validation[seg]=train_arr[32:]

In [20]:
max_min_time

{5: [119.05935933078378, 33.349999999999994],
 3: [139.5, 46.0],
 43: [85.4417573918544, 23.0],
 44: [42.90484734256568, 23.0],
 46: [123.25, 38.0],
 138: [199.10377691912151, 72.6],
 34: [98.13854500484459, 25.0],
 32: [254.5, 47.0],
 297: [133.24436954846146, 44.0],
 316: [183.13197969543145, 36.0],
 59: [90.01347911474551, 46.0],
 71: [104.96755728211456, 40.0],
 57: [284.0, 50.0],
 55: [107.75, 27.0],
 153: [180.25, 48.0],
 152: [172.22564912405616, 68.0],
 221: [141.19167274271376, 71.0],
 385: [168.75, 68.0],
 42: [104.48327849376622, 34.0]}

In [21]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_7\train_dpf.npy',train)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_7\test_dpf.npy',test)

In [12]:
mean_mat={}
for seg in targets:
    tmp=numpy.array(train[seg])
    mean_mat[seg]=numpy.around(numpy.mean(tmp,axis=0),2)

In [12]:
train[5][0].shape

(177,)

## 聚类

In [22]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_7\mean_mat.npy',mean_mat)

In [14]:
mean_mat_slot_20={}
for seg in mean_mat:
    new_record=[numpy.around(numpy.mean(mean_mat[seg][i*12:(i+1)*12]),2) for i in range(175//12)]
    mean_mat_slot_20[seg]=new_record

In [150]:
mean_mat_slot_20_denorm={}
for seg in mean_mat_slot_20:
    max_val,min_val=max_min_time[seg]
    bias=max_val-min_val
    min_arr=numpy.array([min_val for i in range(175//12)])
    mean_mat_slot_20_denorm[seg]=list((numpy.array(mean_mat_slot_20[seg])*bias+min_arr))

In [15]:
mean_mat_slot_20_norm={}
for seg in mean_mat_slot_20:
    max_val=max(mean_mat_slot_20[seg])
    min_val=min(mean_mat_slot_20[seg])
    min_arr=numpy.array([min_val for i in range(175//12)])
    mean_mat_slot_20_norm[seg]=list(numpy.around((mean_mat_slot_20[seg]-min_arr)/(max_val-min_val),2))

In [ ]:
print(mean_mat_slot_20_norm)

In [144]:
numpy.corrcoef(mean_mat_slot_20_norm[54],mean_mat_slot_20_norm[3])

array([[1.        , 0.82204378],
       [0.82204378, 1.        ]])

In [29]:
max_min_speed={}

In [31]:
for seg in targets:
    seg_len=float(info[str(seg)][5])
    max_train,min_train=max_min_time[seg]
    max_min_speed[seg]=[seg_len*3.6/max_train,seg_len*3.6/min_train]

In [58]:
variance_2={}
for seg in target_segs:
    time_arr=numpy.array(train[seg])
    variance_2[seg]=[numpy.around(numpy.mean(time_arr),2),numpy.around(numpy.std(time_arr),2)]

In [59]:
for seg in target_segs:
    time_arr=numpy.array(train[seg])
    day,record=time_arr.shape
    tmp_record=[variance_2[seg][0]]*record
    tmp_day=[tmp_record]*day
    mean=numpy.array(tmp_day)
    tmp_record=[variance_2[seg][1]]*record
    tmp_day=[tmp_record]*day
    std=numpy.array(tmp_day)
    print(seg,numpy.sum(numpy.where(numpy.absolute(time_arr-mean)>3*std,1,0))/day/record)

39 0.03567099567099567
51 0.0
37 0.0
41 0.001038961038961039
43 0.004675324675324675
44 0.03497835497835498
385 0.03722943722943723
139 0.009696969696969697
57 0.0008658008658008658
59 0.0
55 0.04952380952380952
53 0.0


In [50]:
variance_1={}

In [51]:
for seg in target_segs:
    time_arr=numpy.array(train[seg])
    variance_1[seg]=[numpy.around(numpy.mean(time_arr),2),numpy.around(numpy.std(time_arr),2)]

In [52]:
for seg in target_segs:
    time_arr=numpy.array(train[seg])
    day,record=time_arr.shape
    tmp_record=[variance_1[seg][0]]*record
    tmp_day=[tmp_record]*day
    mean=numpy.array(tmp_day)
    tmp_record=[variance_1[seg][1]]*record
    tmp_day=[tmp_record]*day
    std=numpy.array(tmp_day)
    print(seg,numpy.sum(numpy.where(numpy.absolute(time_arr-mean)>3*std,1,0))/day/record)

54 0.0
56 0.0
58 0.0
316 0.022337662337662337
45 0.008658008658008658
246 0.03913419913419913
42 0.03186147186147186
38 0.0
52 0.0
118 0.03722943722943723
112 0.013160173160173162
185 0.0048484848484848485


In [36]:
variance={}

In [37]:
for seg in target_segs:
    time_arr=numpy.array(train[seg])
    # print(numpy.std(speed))
    variance[seg]=[numpy.around(numpy.mean(time_arr),2),numpy.around(numpy.std(time_arr),2)]

In [38]:
variance

{5: [56.85, 12.22],
 3: [69.09, 13.28],
 43: [46.24, 9.4],
 44: [36.14, 3.21],
 46: [74.02, 15.13],
 138: [98.19, 22.07],
 34: [50.07, 12.36],
 32: [96.32, 28.09],
 297: [74.97, 20.81]}

In [39]:
time_arr.shape

(33, 175)

In [40]:
for seg in target_segs:
    time_arr=numpy.array(train[seg])
    day,record=time_arr.shape
    tmp_record=[variance[seg][0]]*record
    tmp_day=[tmp_record]*day
    mean=numpy.array(tmp_day)
    tmp_record=[variance[seg][1]]*record
    tmp_day=[tmp_record]*day
    std=numpy.array(tmp_day)
    print(seg,numpy.sum(numpy.where(numpy.absolute(time_arr-mean)>3*std,1,0))/day/record*100)

5 0.013333333333333334
3 0.0029437229437229437
43 0.004675324675324675
44 0.03497835497835498
46 0.0
138 0.024069264069264067
34 0.0025974025974025974
32 0.004329004329004329
297 0.0


In [ ]:
for seg in target_segs:
    seg_len=float(info[str(seg)][5])
    len_arr=numpy.array([seg_len for i in range(33*175)])
    len_arr=len_arr.reshape((33,175))
    speed=numpy.around(len_arr*3.6/train[seg],2)
    # print(numpy.std(speed))
    variance[seg]=[numpy.around(numpy.mean(speed),2),numpy.around(numpy.std(speed),2)]

In [15]:
for seg in target_segs:
    seg_len=float(info[str(seg)][5])
    min_speed_tolerance=15
    min_speed=20
    max_time_tolerance=seg_len*3.6/min_speed_tolerance
    max_time=seg_len*3.6/min_speed
    test_seg_arr=numpy.array(train[seg])
    print(seg,numpy.sum(numpy.where(test_seg_arr>max_time_tolerance,1,0))/32/180)

    # outliers=numpy.where(test_seg_arr>max_time_tolerance)#查看异常值位置
    # outliers_num=len(outliers[0])

112 0.009548611111111112
185 0.00017361111111111112
186 0.0012152777777777778
192 0.01545138888888889
193 0.009027777777777777
272 0.0
148 0.021527777777777778
188 0.0
217 0.003993055555555555
225 0.0012152777777777778


In [18]:
seg_len=float(info[str(188)][5])
outliers_value=[[seg_len*3.6/train[seg][outliers[0][i]][outliers[1][i]-1],seg_len*3.6/train[seg][outliers[0][i]][outliers[1][i]]] for i in range(outliers_num)]

## 构造输入

In [94]:
#训练集
X={}
for seg,record in train.items():
    input_c=[]
    # input_d=[]
    # input_w=[]
    output=[]
    for day in range(33):
        valid=record[day] # a list
        for i in range(171):
            input_c.append(valid[i:i+4])#前4个时间步
            output.append([valid[i+4]])
            # tmp_d=[record[day-step][i] for step in range(4)] #前四天
            # tmp_w=[record[(day-step*7)*180+i] for step in range(1)]#前一周
            # input_d.append(tmp_d)
            # input_w.append(tmp_w)
    X[seg]=[input_c,output]
    # train_seg[seg]=[input_c,input_d,output]
    # train_seg[seg]=[input_c,input_d,input_w,output]


In [ ]:
train_days,train_dim=train[5].shape

In [72]:
len(train_seg[5][0])

5643

In [76]:
input_x=X[targets[0]][1]
for seg in targets[1:]:
    input_c=X[seg][1]
    input_x=numpy.concatenate((input_x,input_c),axis=1)


In [77]:
input_0.shape

(5643, 36)

In [95]:
output_x=X[targets[0]][1]
for seg in targets[1:]:
    output=X[seg][1]
    output_x=numpy.concatenate((output_x,output),axis=1)

In [96]:
output_0.shape

(5643, 9)

In [ ]:
def get_input(targets):
    train_flat=numpy.array([numpy.ravel(train[seg]) for seg in targets])
    train_time_steps=numpy.array([train_flat[:,time_step] for time_step in range(train_flat.shape[1])])
    test_flat=numpy.array([numpy.ravel(test[seg]) for seg in targets])
    test_time_steps=numpy.array([test_flat[:,time_step] for time_step in range(test_flat.shape[1])])
    train_in_c=[]
    train_out_c=[]
    train_in_d=[]
    for day in range(4,train_days):
        day_base=train_time_steps[day*train_dim:(day+1)*train_dim]
        for i in range(train_dim-look_back_span):
            train_in_c.append(day_base[i:i+look_back_span])
            train_out_c.append(day_base[i+look_back_span]) 
            train_in_d.append([train_time_steps[(day-i)*train_dim+i] for i in range(look_back_days)])
    test_in_c=[]
    test_out_c=[]
    test_in_d=[]
    for day in range(test_days):
        day_base=test_time_steps[day*test_dim:(day+1)*test_dim]
        for i in range(test_dim-look_back_span):
            test_in_c.append(day_base[i:i+look_back_span])
            test_out_c.append(day_base[i+look_back_span]) 
            test_in_d.append([test_time_steps[(day-i)*train_dim+i] for i in range(look_back_days)])
    train_in_c=numpy.array(train_in_c)
    train_out_c=numpy.array(train_out_c)
    test_in_c=numpy.array(test_in_c)
    test_out_c=numpy.array(test_out_c)
    train_in_d=numpy.array(train_in_d)
    test_in_d=numpy.array(test_in_d)
    return train_in_c,test_in_c,train_out_c,test_out_c,train_in_d,test_in_d

In [103]:
#测试集
Y={}
for seg,record in test.items():
    input_c=[]
    input_d=[]
    input_w=[]
    output=[]
    for day in range(8):
        valid=record[day] # a list
        for i in range(176):
            input_c.append(valid[i:i+4])
            output.append([valid[i+4]])
            # tmp_d=[record[day-step][i] for step in range(4)] #前四天
            # tmp_w=[record[(day-step*7)*180+i] for step in range(1)]#前一周
            # input_d.append(tmp_d)
            # input_w.append(tmp_w)
    # Y[seg]=[input_c,input_d,input_w,output]
    # Y[seg]=[input_c,input_d,output]
    Y[seg]=[input_c,output]

In [ ]:
#测试集
Y={}
for seg,record in test.items():
    input_c=[]
    input_d=[]
    input_w=[]
    output=[]
    for day in range(8):
        valid=record[day] # a list
        for i in range(176):
            input_c.append(valid[i:i+4])
            output.append([valid[i+4]])
    Y[seg]=[input_c,output]

In [85]:
input_y=Y[targets[0]][0]
for seg in targets[1:]:
    input_c=Y[seg][0]
    input_y=numpy.concatenate((input_y,input_c),axis=1)

In [97]:
input_1.shape

(1408, 36)

In [104]:
output_y=Y[targets[0]][1]
for seg in targets[1:]:
    output_c=Y[seg][1]
    output_y=numpy.concatenate((output_y,output_c),axis=1)

In [105]:
output_1.shape

(1408, 9)

In [107]:
output_0.shape

(5643, 9)

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_6\train_input.npy',[input_0])


In [110]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_6\train_input.npy',input_x)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_6\train_output.npy',output_x)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_6\test_input.npy',input_y)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_6\test_output.npy',output_y)

In [41]:
#验证集
validation_seg={}
for seg,record in validation.items():
    input_c=[]
    input_d=[]
    input_w=[]
    output=[]
    for day in range(4,12):
        valid=record[day] # a list
        for i in range(174):
            input_c.append(valid[i:i+4])
            output.append(valid[i+4])
            # tmp_d=[record[(day-step)*180+i] for step in range(2)] #前四天
            tmp_d=[record[day-step][i] for step in range(4)] #前四天

            # tmp_w=[record[(day-step*7)*180+i] for step in range(1)]#前一周
            input_d.append(tmp_d)
            # input_w.append(tmp_w)
    validation_seg[seg]=[input_c,input_d,output]

In [44]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_5\train.npy',train_seg)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_5\test.npy',test_seg)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_5\validation.npy',validation_seg)

In [ ]:
time_stamp=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\time_stamp.npy',allow_pickle=True).item()
ts_list=list(time_stamp.keys())

In [ ]:
ts=[]
for each in ts_list:
    tmp=each.split(':')
    ts.append(tmp[0]+tmp[1])

In [ ]:
ts[:5]

In [ ]:
new_ts=[]
for i in range(180):
    new_ts.append(ts[i*5])

In [ ]:
from keras.preprocessing.text import one_hot

In [ ]:
encoded_time_stamp=[]
for i in new_ts:
    encoded_time_stamp+=one_hot(i,180)

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\encoded_time_stamp.npy',encoded_time_stamp)

In [5]:
dates_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\收集日期对应工作日.npy',allow_pickle=True).item()

In [8]:
for date in dates_dict.keys():
    dates_dict[date]-=1

In [10]:
numpy.save(r'C:\Users\e-wel\Desktop\day_of_week.npy',dates_dict)

In [ ]:
test_day_of_week=[]
for i in day_of_week:
    test_day_of_week+=[dates_dict[i] for j in range(174)]
    # test_day_of_week+=numpy.repeat([dates_dict[i]],176,axis=0)

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\validate_day_of_week.npy',test_day_of_week)

## Deep PF 输入准备

In [13]:
test_index=[i for i in range(46,60)]
train_index=[i for i in range(46)]

In [15]:
# targets=[5, 3, 43, 44,46, 138, 34, 32, 297]
targets=[5, 3, 43, 44,46, 138, 34, 32, 297]+[316,59,71,57,55,153,152,221,385,42]
train={}
test={}
for seg in targets:
    train[seg]=[total[seg][i] for i in train_index]
    test[seg]=[total[seg][i] for i in test_index]

In [16]:
len(train[5]),len(test[seg])

(46, 14)